In [1]:
import json
import warnings
from typing import Any, Literal

import numpy as np
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)


# Demo (Prevents ruff from removing the unused module import)
name: Any
category: Literal["A", "B", "C"]
json.loads('{"name": "Bike Rental Prediction", "category": "A"}')

{'name': 'Bike Rental Prediction', 'category': 'A'}

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/Projects/Bike-Rental-Prediction


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
import narwhals as nw
from narwhals.typing import IntoDataFrameT


def split_temporal_data_to_train_val_test(
    data: IntoDataFrameT,
    test_size: float = 0.2,
    val_size: float = 0.2,
    print_shapes: bool = True,
) -> tuple[IntoDataFrameT, IntoDataFrameT, IntoDataFrameT]:
    """Split data into train, validation and test sets while maintaining temporal order.

    Parameters
    ----------
    data : IntoDataFrameT
        The input dataframe to be split.
    test_size : float, default=0.2
        Proportion of the dataset to include in the test split.
    val_size : float, default=0.2
        Proportion of the dataset to include in the validation split.
    print_shapes : bool, default=True
        Whether to print the shapes of the resulting dataframes.

    Returns
    -------
    tuple[IntoDataFrameT, IntoDataFrameT, IntoDataFrameT]
        A tuple containing the train, validation, and test dataframes.
    """
    # Validation
    if not 0.0 <= test_size <= 1.0 or not 0.0 <= val_size <= 1.0:
        raise ValueError("test_size and val_size must be between 0.0 and 1.0")
    if (test_size + val_size) >= 1.0:
        raise ValueError("The sum of test_size and val_size must be less than 1.0")

    # Convert to Narwhals DataFrame
    nw_data: nw.DataFrame = nw.from_native(data)
    num_train: int = int((1 - test_size - val_size) * nw_data.shape[0])
    num_val: int = int(val_size * nw_data.shape[0])

    # Add index
    nw_data = nw_data.with_row_index()
    train_data: nw.DataFrame = nw_data.filter(nw.col("index") <= num_train).drop(
        "index"
    )

    val_data: nw.DataFrame = nw_data.filter(
        (nw.col("index") > num_train) & (nw.col("index") <= (num_train + num_val))
    ).drop("index")
    test_data: nw.DataFrame = nw_data.filter(
        nw.col("index") > (num_train + num_val)
    ).drop("index")

    if print_shapes:
        print(
            f"Train shape: {train_data.shape} | Val shape: {val_data.shape} | Test shape: {test_data.shape}"
        )

    return (train_data.to_native(), val_data.to_native(), test_data.to_native())

In [6]:
df: pl.DataFrame = pl.DataFrame(
    {
        "idx": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
        "role": [
            "engineer",
            "doctor",
            "nurse",
            "engineer",
            "nurse",
            "teacher",
            "doctor",
            "doctor",
            "teacher",
            "engineer",
            "teacher",
            "nurse",
            "teacher",
            "engineer",
            "nurse",
        ],
    }
)

df

idx,role
i64,str
0,"""engineer"""
1,"""doctor"""
2,"""nurse"""
3,"""engineer"""
4,"""nurse"""
5,"""teacher"""
6,"""doctor"""
7,"""doctor"""
8,"""teacher"""


In [7]:
(train_df, val_df, test_df) = split_temporal_data_to_train_val_test(
    data=df,
    test_size=0.8,
    val_size=0.1,
)
train_df

Train shape: (2, 2) | Val shape: (1, 2) | Test shape: (12, 2)


idx,role
i64,str
0,"""engineer"""
1,"""doctor"""


In [8]:
rng = np.random.default_rng(123)
x = rng.standard_normal(size=(1_000, 10))

X_train, X_test = train_test_split(x, test_size=0.2, random_state=123)
y_train = rng.standard_normal(size=(X_train.shape[0],))
y_test = rng.standard_normal(size=(X_test.shape[0],))

params: dict[str, Any] = {
    "n_estimators": 100,
    "max_depth": 10,
}

rf_reg = RandomForestRegressor(**params)

rf_reg.fit(X_train, y_train)
# rf_reg.score(X_test, y_test)

,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [9]:
rf_reg.feature_importances_

array([0.1154, 0.0987, 0.0682, 0.109 , 0.089 , 0.1209, 0.1017, 0.1034,
       0.0989, 0.0948])

In [10]:
from src.config import app_config
from src.exceptions import (
    CustomError,
    MLFlowConnectionError,
)
from src.ml.feature_engineering import FeatureEngineer
from src.ml.trainer import ModelTrainer
from src.ml.utils import split_temporal_data_to_train_val_test

In [ ]:
import httpx
import mlflow
import mlflow.sklearn

port: int = 5001
url: str = f"http://localhost:{port}"


def check_mlflow(url: str, timeout: float = 2.0) -> bool:
    """
    Check MLflow endpoint accessibility and handle common httpx errors.

    Returns True if reachable (2xx), False otherwise.
    """
    try:
        resp = httpx.get(url, timeout=timeout)
        resp.raise_for_status()
        console.print("[success]MLflow is accessible")
        return True

    except httpx.HTTPStatusError as e:
        console.print(
            f"[error]MLflow returned non-2xx status: {e.response.status_code} — {e}"
        )
        return False

    except httpx.RequestError as e:
        # covers ConnectError, ReadTimeout, etc.
        console.print(f"[error]Network/connection error when contacting MLflow: {e}")
        return False

    except (MLFlowConnectionError, CustomError) as e:
        console.print(f"[error]Project-specific MLflow error: {e}")
        return False

    except Exception as e:
        console.print(f"[error]Unexpected error: {e}")
        return False


check_mlflow(url)

MLflow is accessible

True

In [12]:
fp: str = "../../../../Documents/data_dump/bike_data/database.parquet"
data: pl.DataFrame = pl.read_parquet(fp)
console.print(f"Shape: {data.shape}", style="info")
display(data.head())

(train_df, val_df, test_df) = split_temporal_data_to_train_val_test(data=data)

Shape: (13903, 16)

datetime,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64
"""2011-01-01 00:00:00""",1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
"""2011-01-01 01:00:00""",1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0.0,8,32,40
"""2011-01-01 02:00:00""",1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0.0,5,27,32
"""2011-01-01 03:00:00""",1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
"""2011-01-01 04:00:00""",1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Shapes -> Train shape: (8342, 16) | Val shape: (2780, 16) | Test shape: (2781, 16)


In [43]:
train_df.sample(2).to_dicts()

# pl.from_records(train_df.sample().to_dicts())

[{'datetime': '2011-05-23 09:00:00',
  'season': 2,
  'yr': 0,
  'mnth': 5,
  'hr': 9,
  'holiday': 0,
  'weekday': 1,
  'workingday': 1,
  'weathersit': 1,
  'temp': 0.6,
  'atemp': 0.5455,
  'hum': 0.88,
  'windspeed': 0.2239,
  'casual': 36,
  'registered': 131,
  'cnt': 167},
 {'datetime': '2011-09-29 09:00:00',
  'season': 4,
  'yr': 0,
  'mnth': 9,
  'hr': 9,
  'holiday': 0,
  'weekday': 4,
  'workingday': 1,
  'weathersit': 1,
  'temp': 0.62,
  'atemp': 0.5909,
  'hum': 0.78,
  'windspeed': 0.2537,
  'casual': 27,
  'registered': 198,
  'cnt': 225}]

Peak hour price: $884.54
Off-peak price: $610.3
Average conditions price: $758.77

Batch predictions: [884.54, 610.3, 758.77]

Premium pricing (800-1500): $1338.35


In [ ]:
from src.exp_tracking.model_loader import (
    get_best_run,
    load_best_model,
)

In [ ]:
feat_eng = FeatureEngineer()
train_features_df: pl.DataFrame = feat_eng.transform(
    data=train_df, config=app_config.feature_config
)
val_features_df: pl.DataFrame = feat_eng.transform(
    data=val_df, config=app_config.feature_config
)
test_features_df: pl.DataFrame = feat_eng.transform(
    data=test_df, config=app_config.feature_config
)
display(train_features_df.head())

val_features_df.head()

In [ ]:
trainer = ModelTrainer(
    train_data=train_features_df,
    val_data=val_features_df,
    test_data=test_features_df,
    target_col="target",
)

In [ ]:
get_best_run(
    experiment_name=app_config.experiment_config.experiment_name,
    client=trainer.mlflow_tracker.client,
)

In [ ]:
best_model = load_best_model(
    experiment_name=app_config.experiment_config.experiment_name,
    client=trainer.mlflow_tracker.client,
)

In [ ]:
import mlflow

try:
    mlflow.end_run()
except Exception as e:
    print(e)

In [ ]:
# trainer._hyperparameter_tuning_xgboost()
# result = trainer._hyperparameter_tuning_lightgbm()

# trainer.hyperparameter_tuning_all_models()
result = trainer.train_all_models()

In [ ]:
# console.print(result)

In [ ]:
# 25
col: str = "windspeed"
train_df[col].describe().to_dicts()

In [ ]:
{
    "data_shape": {
        "total_rows": 13903,
        "total_columns": 16,
        "number_of_numeric_columns": 15,
        "number_of_categorical_columns": 1,
    },
    "other_info": {
        "data_nulls": {
            "hr": 0,
            "yr": 0,
            "cnt": 0,
            "hum": 0,
            "mnth": 0,
            "temp": 0,
            "atemp": 0,
            "casual": 0,
            "season": 0,
            "holiday": 0,
            "weekday": 0,
            "datetime": 0,
            "windspeed": 0,
            "registered": 0,
            "weathersit": 0,
            "workingday": 0,
        },
        "total_nulls": 0,
        "memory_usage_MB": 1.84,
        "num_duplicated_rows": 0,
        "validation_timestamp": "2025-10-14T12:58:00+00:00",
        "num_unique_numeric_rows": {
            "hr": 24,
            "yr": 2,
            "cnt": 789,
            "hum": 88,
            "mnth": 12,
            "temp": 50,
            "atemp": 65,
            "casual": 303,
            "season": 4,
            "holiday": 2,
            "weekday": 7,
            "windspeed": 30,
            "registered": 701,
            "weathersit": 4,
            "workingday": 2,
        },
        "num_unique_categorical_rows": {"datetime": 13903},
    },
    "data_schema": {
        "numeric": {
            "hr": "Int64",
            "yr": "Int64",
            "cnt": "Int64",
            "hum": "Float64",
            "mnth": "Int64",
            "temp": "Float64",
            "atemp": "Float64",
            "casual": "Int64",
            "season": "Int64",
            "holiday": "Int64",
            "weekday": "Int64",
            "windspeed": "Float64",
            "registered": "Int64",
            "weathersit": "Int64",
            "workingday": "Int64",
        },
        "categorical": {},
    },
    "summary_statistics": {
        "numeric": [
            {
                "max": 4.0,
                "min": 1.0,
                "std": 1.04,
                "mean": 2.26,
                "mode": [2.0],
                "count": 13903,
                "range": 3.0,
                "column": "season",
                "median": 2.0,
                "variance": 1.07,
                "missing_pct": 0.0,
                "unique_values": 4,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.0,
                "std": 0.48,
                "mean": 0.38,
                "mode": [0.0],
                "count": 13903,
                "range": 1.0,
                "column": "yr",
                "median": 0.0,
                "variance": 0.24,
                "missing_pct": 0.0,
                "unique_values": 2,
                "missing_values": 0,
            },
            {
                "max": 12.0,
                "min": 1.0,
                "std": 3.22,
                "mean": 5.65,
                "mode": [7.0, 5.0],
                "count": 13903,
                "range": 11.0,
                "column": "mnth",
                "median": 5.0,
                "variance": 10.36,
                "missing_pct": 0.0,
                "unique_values": 12,
                "missing_values": 0,
            },
            {
                "max": 23.0,
                "min": 0.0,
                "std": 6.91,
                "mean": 11.55,
                "mode": [16.0, 17.0],
                "count": 13903,
                "range": 23.0,
                "column": "hr",
                "median": 12.0,
                "variance": 47.78,
                "missing_pct": 0.0,
                "unique_values": 24,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.0,
                "std": 0.16,
                "mean": 0.03,
                "mode": [0.0],
                "count": 13903,
                "range": 1.0,
                "column": "holiday",
                "median": 0.0,
                "variance": 0.03,
                "missing_pct": 0.0,
                "unique_values": 2,
                "missing_values": 0,
            },
            {
                "max": 6.0,
                "min": 0.0,
                "std": 2.01,
                "mean": 3.0,
                "mode": [6.0],
                "count": 13903,
                "range": 6.0,
                "column": "weekday",
                "median": 3.0,
                "variance": 4.02,
                "missing_pct": 0.0,
                "unique_values": 7,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.0,
                "std": 0.46,
                "mean": 0.68,
                "mode": [1.0],
                "count": 13903,
                "range": 1.0,
                "column": "workingday",
                "median": 1.0,
                "variance": 0.22,
                "missing_pct": 0.0,
                "unique_values": 2,
                "missing_values": 0,
            },
            {
                "max": 4.0,
                "min": 1.0,
                "std": 0.64,
                "mean": 1.42,
                "mode": [1.0],
                "count": 13903,
                "range": 3.0,
                "column": "weathersit",
                "median": 1.0,
                "variance": 0.41,
                "missing_pct": 0.0,
                "unique_values": 4,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.02,
                "std": 0.2,
                "mean": 0.5,
                "mode": [0.62],
                "count": 13903,
                "range": 0.98,
                "column": "temp",
                "median": 0.5,
                "variance": 0.04,
                "missing_pct": 0.0,
                "unique_values": 50,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.0,
                "std": 0.18,
                "mean": 0.48,
                "mode": [0.62],
                "count": 13903,
                "range": 1.0,
                "column": "atemp",
                "median": 0.48,
                "variance": 0.03,
                "missing_pct": 0.0,
                "unique_values": 65,
                "missing_values": 0,
            },
            {
                "max": 1.0,
                "min": 0.0,
                "std": 0.2,
                "mean": 0.62,
                "mode": [0.88],
                "count": 13903,
                "range": 1.0,
                "column": "hum",
                "median": 0.62,
                "variance": 0.04,
                "missing_pct": 0.0,
                "unique_values": 88,
                "missing_values": 0,
            },
            {
                "max": 0.85,
                "min": 0.0,
                "std": 0.12,
                "mean": 0.19,
                "mode": [0.0],
                "count": 13903,
                "range": 0.85,
                "column": "windspeed",
                "median": 0.19,
                "variance": 0.02,
                "missing_pct": 0.0,
                "unique_values": 30,
                "missing_values": 0,
            },
            {
                "max": 367.0,
                "min": 0.0,
                "std": 47.34,
                "mean": 34.04,
                "mode": [0.0],
                "count": 13903,
                "range": 367.0,
                "column": "casual",
                "median": 15.0,
                "variance": 2241.19,
                "missing_pct": 0.0,
                "unique_values": 303,
                "missing_values": 0,
            },
            {
                "max": 796.0,
                "min": 0.0,
                "std": 137.51,
                "mean": 140.6,
                "mode": [4.0],
                "count": 13903,
                "range": 796.0,
                "column": "registered",
                "median": 107.0,
                "variance": 18909.65,
                "missing_pct": 0.0,
                "unique_values": 701,
                "missing_values": 0,
            },
            {
                "max": 957.0,
                "min": 1.0,
                "std": 166.96,
                "mean": 174.64,
                "mode": [5.0],
                "count": 13903,
                "range": 956.0,
                "column": "cnt",
                "median": 130.0,
                "variance": 27874.11,
                "missing_pct": 0.0,
                "unique_values": 789,
                "missing_values": 0,
            },
        ],
        "categorical": [
            {
                "column": "datetime",
                "missing_pct": 0.0,
                "total_count": 13903,
                "value_counts": [
                    {
                        "__data__": ["2011-01-01 00:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 01:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 02:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 03:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 04:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 05:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 06:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 07:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 08:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                    {
                        "__data__": ["2011-01-01 09:00:00", 1],
                        "__version__": 1,
                        "__classname__": "builtins.tuple",
                    },
                ],
                "unique_values": 13903,
                "missing_values": 0,
            }
        ],
    },
}